In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
import tensorflow as tf
import numpy as np
import pandas as pd

2025-05-28 17:09:10.981213: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 17:09:11.233881: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 17:09:11.451453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748473751.632483    8789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748473751.680243    8789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748473752.089836    8789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# load data
df = pd.read_csv("../csv/numbers.csv")
df.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(10000, 785)

In [4]:
# Preparar datos
def one_hot(n_value, num_classes=10):
    l_arr = np.zeros(num_classes)
    if 0 <= int(n_value) < num_classes:
        l_arr[int(n_value)] = 1
    else:
        print(f"Advertencia: Valor de etiqueta {n_value} fuera de rango para {num_classes} clases.")
    return l_arr

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Y_raw = df.iloc[:, 0].values # todas las filas [:] columna 0 (labels) [:, 0]
X_raw = df.iloc[:, 1:].values # todas las filas [:] colunas de la 1 en adelante (datos) [:, 1:]
Y_processed = np.array([one_hot(n) for n in Y_raw]).astype('float32')
x_train = x_train.astype('float32') / 255.0 # para normalizar los datos 0-1
y_train = np.array([one_hot(n) for n in y_train]).astype('float32')
X_normalized = X_raw.astype('float32') / 255.0 # para normalizar los datos 0-1

x_train = x_train.reshape(60000, 784)
X_normalized.shape, Y_processed.shape, x_train.shape, y_train.shape



((10000, 784), (10000, 10), (60000, 784), (60000, 10))

In [5]:
# Crear Modelo
model = tf.keras.Sequential(name="myMnist")
model.add(tf.keras.layers.InputLayer(shape=(784,)))
model.add(tf.keras.layers.Dense(150, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Resumen del modelo
model.summary()


2025-05-28 17:09:18.030337: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "myMnist"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 150)            │       117,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,260 (465.86 KB)

 Trainable params: 119,260 (465.86 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Entrenar modelo
print("\nEntrenando con Keras...")
pasos = 80
batch = 32
# history = model.fit(X_normalized, Y_processed, epochs=pasos, batch_size=batch, verbose=1)
history = model.fit(x_train, y_train, epochs=pasos, batch_size=batch, verbose=1)

print(f"Pérdida final: {history.history['loss'][-1]:.4f}")
print(f"Accuracy final: {history.history['accuracy'][-1]:.4f}")

# Puedes guardar y cargar el modelo fácilmente
model.save("mnist.h5")
# loaded_model = tf.keras.models.load_model("mi_modelo_keras.h5")


Entrenando con Keras...
Epoch 1/80


2025-05-28 17:09:18.311867: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8965 - loss: 0.3336
Epoch 2/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9558 - loss: 0.1566
Epoch 3/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9643 - loss: 0.1269
Epoch 4/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9679 - loss: 0.1207
Epoch 5/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9704 - loss: 0.1080
Epoch 6/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9717 - loss: 0.1062
Epoch 7/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9717 - loss: 0.1073
Epoch 8/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9745 - loss: 0.0972
Epoch 9/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9760 - loss: 0.0920
Epoch 10/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9768 - loss: 0.0934
Epoch 11/80
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9789 - loss: 0.0840
Epoch 12/80
1875/1875 ━━━━━━━━━━━━━━━━━━━

Pérdida final: 0.0482
Accuracy final: 0.9930
